このノートブックでは, これまでに定義した関数を用いてOptunaによるハイパーパラメータ最適化の結果をMLFlow trackingで記録します.

In [1]:
from typing import Any, Dict, Tuple, List
import numpy as np
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from pandas import DataFrame
import optuna
import mlflow

df = sns.load_dataset('titanic')
df.head()

# 必要な特徴量を抽出
feature_names = [
    'class',
    'sex',
    'age',
    'sibsp',
    'parch',
    'fare',
    'embark_town',
    'deck',
]
df_x = df[feature_names]
df_y = df['survived']

class IntOrdEncoder(ce.OrdinalEncoder):
    def __init__(self, cols, mapping, handle_unknown):
        super().__init__(cols=cols, mapping=mapping, handle_unknown=handle_unknown)
        self.cols = cols

    def transform(self, *args, **kwargs):
        """xはpd.DataFrameです.
        """
        x = super().transform(*args, **kwargs)
        for col in self.cols:
            x[col] = x[col].astype(int)

        return x

    def fit_transform(self, *args, **kwargs):
        """xはpd.DataFrameです.
        """
        x = super().fit_transform(*args, **kwargs)
        for col in self.cols:
            x[col] = x[col].astype(int)

        return x


x_tr, x_te, y_tr, y_te = train_test_split(df_x, df_y, test_size=0.33, shuffle=True, random_state=42)


def titanic_cat_encoder() -> Tuple[List[str], ce.OrdinalEncoder]:
    cols = ['class', 'sex', 'embark_town', 'deck']
    mapping = [
        {"col": "class", "mapping": {"First": 0, "Second": 1, "Third": 2}},
        {"col": "sex", "mapping": {"male": 0, "female": 1}},
        {"col": "embark_town", "mapping": {"Southampton": 0, "Cherbourg": 1, "Queenstown": 2}},
        {"col": "deck", "mapping": {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}},
    ]
    return (cols, IntOrdEncoder(cols=cols, mapping=mapping, handle_unknown='value'))


def fit_eval(x, y, encoder, params, cols, nested):
    with mlflow.start_run(nested=nested):
        mlflow.log_param("depth", params["depth"])
        mlflow.log_param("learning_rate", params["learning_rate"])
        mlflow.log_param("random_strength", params["random_strength"])
        mlflow.log_param("l2_leaf_reg", params["l2_leaf_reg"])

        # # cat_featuresを正しい値に設定
        # cat_features = None if params["cat_features"] == "none" else cols
        # params["cat_features"] = cat_features
        params["cat_features"] = ["embark_town", "deck"]

        # パイプラインを構成
        clf = CatBoostClassifier(
            **params,
            verbose=False
        )
        pipe = make_pipeline(encoder, clf)

        # スコアを計算
        score = cross_val_score(pipe, x, y, cv=5).mean()
        mlflow.log_metric("cv_score", score)

        return score


def suggest_params(trial: optuna.Trial) -> Dict:
    return {
        "depth": trial.suggest_int("depth", 1, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", np.exp(-7.0), 1.0),
        "random_strength": trial.suggest_int("random_strength", 1, 20),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1, 10),
    }

/Users/taku-y/github/PyDataOsaka/learn-mlflow/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Optunaで利用するための目的関数を定義します. 既に下位のRunで訓練と評価を行いログを取る関数を定義しているので, ハイパーパラメータに対する評価を返す関数を定義します.

In [2]:
def create_objective(x: DataFrame, y: DataFrame) -> Any:  # 戻り値の型は後で書く
    (cols, encoder) = titanic_cat_encoder()

    def objective(trial: optuna.Trial) -> Any:  # 戻り値の型は後で書く
        params = suggest_params(trial)

        return fit_eval(x, y, encoder, params, cols, nested=True)

    return objective

結果の中から最適なハイパーパラメータを記録する関数を定義します.

In [3]:
def log_best(run: mlflow.entities.Run, metric: str="cv_score") -> Dict:
    client = mlflow.tracking.MlflowClient()
    runs = client.search_runs(
        [run.info.experiment_id],
        "tags.mlflow.parentRunId = '{run_id}' ".format(run_id=run.info.run_id)
    )

    best_run = max(runs, key=lambda run: run.data.metrics[metric])

    mlflow.set_tag("best_run", best_run.info.run_id)
    mlflow.log_metric(f"best_{metric}", best_run.data.metrics[metric])

    return best_run.data.params

あとはMLFlowの上位のRunを開始し, この関数をOptunaに与えます.

In [4]:
with mlflow.start_run() as run:
    objective = create_objective(x_tr, y_tr)
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=200, show_progress_bar=True)
    params = log_best(run)
    print(params)
    


[I 2022-08-21 13:59:26,597] A new study created in memory with name: no-name-61bd0b6e-f9df-4bc2-a424-c3d6762efc0b
/Users/taku-y/github/PyDataOsaka/learn-mlflow/venv/lib/python3.9/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|          | 1/200 [00:12<39:53, 12.03s/it]

[I 2022-08-21 13:59:38,669] Trial 0 finished with value: 0.8188095238095239 and parameters: {'depth': 8, 'learning_rate': 0.0033935879246515333, 'random_strength': 15, 'l2_leaf_reg': 1.8593891697502458}. Best is trial 0 with value: 0.8188095238095239.


  1%|          | 2/200 [00:24<39:49, 12.07s/it]

[I 2022-08-21 13:59:50,772] Trial 1 finished with value: 0.8171988795518208 and parameters: {'depth': 6, 'learning_rate': 0.2852472648037959, 'random_strength': 7, 'l2_leaf_reg': 8.726408371144352}. Best is trial 0 with value: 0.8188095238095239.


  2%|▏         | 3/200 [00:33<36:17, 11.05s/it]

...

[I 2022-08-21 14:35:05,054] Trial 197 finished with value: 0.8155042016806723 and parameters: {'depth': 8, 'learning_rate': 0.03191721686221072, 'random_strength': 11, 'l2_leaf_reg': 1.0451677858609174}. Best is trial 95 with value: 0.8288935574229692.


100%|█████████▉| 199/200 [35:46<00:09,  9.83s/it]

[I 2022-08-21 14:35:13,449] Trial 198 finished with value: 0.8155042016806723 and parameters: {'depth': 7, 'learning_rate': 0.07632396418239634, 'random_strength': 14, 'l2_leaf_reg': 1.7144931579144895}. Best is trial 95 with value: 0.8288935574229692.


100%|██████████| 200/200 [35:53<00:00, 10.77s/it]


[I 2022-08-21 14:35:20,066] Trial 199 finished with value: 0.8070868347338935 and parameters: {'depth': 6, 'learning_rate': 0.027596510883966502, 'random_strength': 12, 'l2_leaf_reg': 1.3495108074068614}. Best is trial 95 with value: 0.8288935574229692.
{'depth': '7', 'l2_leaf_reg': '3.7763820094293927', 'learning_rate': '0.0655449752625249', 'random_strength': '10'}


先ほどのノートブックと同様にしてMLFlow tracking UIで結果を可視化できます.